# Connecting to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')

import numpy as np
import matplotlib.pyplot as plt
from rockpool.devices.xylo.syns65302 import config_from_specification as config_from, mapper as mapper, XyloSamna as XyloSamna, XyloMonitor as XyloMonitor
from rockpool.devices.xylo.syns65302 import xa3_devkit_utils as hdu

from rockpool.transform.quantize_methods import channel_quantize

Samna version 0.39.11.7+g04f769652


In [2]:
# - Getting the connected devices and choosing xyloa3 board
xylo_node = hdu.find_xylo_a3_boards()[0]

print(xylo_node)


## Clock frequencies

In [3]:
clock_frequencies = [6.25, 12.5, 25, 50]

## Configuration to read registers

In [4]:
import samna
print(f"samna version: {samna.__version__}")
import time

# set device configuration
ioc = xylo_node.get_io_control_module()
io = xylo_node.get_io_module()
model = xylo_node.get_model()


def read_register(addr):
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

ctrl1 = 0x0001
ctrl2 = 0x0002
ctrl3 = 0x0003
tr_wrap = 0x0004
hm_tr_wrap = 0x0005
clk_ctrl = 0x0006
clk_div = 0x0007
pwr_ctrl1 = 0x0008
pwr_ctrl2 = 0x0009
pwr_ctrl3 = 0x000A
pwr_ctrl4 = 0x000B
pad_ctrl = 0x000C
ie1 = 0x000E
ie2 = 0x000F
out_ctrl = 0x0011
monsel = 0x0166
mon_grp_sel = 0x0167
dbg_ctrl1 = 0x0168
dbg_stat1 = 0x0171
dfe_ctrl = 0x001B
ivgen = 0x0015
ivgen2 = 0x0016
ivgen3 = 0x0017
ivgen4 = 0x0018
ivgen5 = 0x0019
ivgen6 = 0x001A
adctest = 0x016E

# create access to xylo input/output
source = samna.graph.source_to(model.get_sink_node())
sink   = samna.graph.sink_from(model.get_source_node())


# print("\n# ======= read important registers =======\n")

# print(f"adctest       : 0x{format(read_register(adctest),'_X')}")
# print(f"ivgen       : 0x{format(read_register(ivgen),'_X')}")
# print(f"ivgen2       : 0x{format(read_register(ivgen2),'_X')}")
# print(f"ivgen3       : 0x{format(read_register(ivgen3),'_X')}")
# print(f"ivgen4       : 0x{format(read_register(ivgen4),'_X')}")
# print(f"ivgen5       : 0x{format(read_register(ivgen5),'_X')}")
# print(f"ivgen6       : 0x{format(read_register(ivgen6),'_X')}")
# # # ctrl[1-3]
# print(f"ctrl1       : 0x{format(read_register(ctrl1),'_X')}")
# print(f"ctrl2       : 0x{format(read_register(ctrl2),'_X')}")
# print(f"ctrl3       : 0x{format(read_register(ctrl3),'_X')}")
# print()
# # *_wrap
# print(f"tr_wrap     : 0x{format(read_register(tr_wrap),'_X')}")
# print(f"hm_tr_wrap  : 0x{format(read_register(hm_tr_wrap),'_X')}")
# print()
# # clk_*
# print(f"clk_ctrl    : 0x{format(read_register(clk_ctrl),'_X')}")
# print(f"clk_div     : 0x{format(read_register(clk_div),'_X')}")
# print()
# # pwr_ctrl[1-4]
# print(f"pwr_ctrl1   : 0x{format(read_register(pwr_ctrl1),'_X')}")
# print(f"pwr_ctrl2   : 0x{format(read_register(pwr_ctrl2),'_X')}")
# print(f"pwr_ctrl3   : 0x{format(read_register(pwr_ctrl3),'_X')}")
# print(f"pwr_ctrl4   : 0x{format(read_register(pwr_ctrl4),'_X')}")
# print()
# # misc
# print(f"pad_ctrl    : 0x{format(read_register(pad_ctrl),'_X')}")
# print(f"ie1         : 0x{format(read_register(ie1),'_X')}")
# print(f"ie2         : 0x{format(read_register(ie2),'_X')}")
# print(f"out_ctrl    : 0x{format(read_register(out_ctrl),'_X')}")
# print()
# # debug regs
# print(f"monsel      : 0x{format(read_register(monsel),'_X')}")
# print(f"mon_grp_sel : 0x{format(read_register(mon_grp_sel),'_X')}")
# print(f"dbg_ctrl1   : 0x{format(read_register(dbg_ctrl1),'_X')}")
# print(f"dbg_stat1   : 0x{format(read_register(dbg_stat1),'_X')}")
# print(f"dfe_ctrl   : 0x{format(read_register(dfe_ctrl),'_X')}")


samna version: 0.39.11.7+g04f769652


# Power measurement - idle - no network

In [ ]:
dt = 0.01 # seconds
duration = 3 # seconds

T = int(duration / dt) # timesteps

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three second in silence
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=None, power_frequency=100, dt=dt, output_mode='Spike')

    sink.clear_events()
    # print(clock)
    # print(f"tr_wrap     : 0x{format(read_register(tr_wrap),'_X')}")
    # pwr_ctrl[1-4]
    print(f"pwr_ctrl1   : 0x{format(read_register(pwr_ctrl1),'_X')}")
    print(f"pwr_ctrl2   : 0x{format(read_register(pwr_ctrl2),'_X')}")
    print(f"pwr_ctrl3   : 0x{format(read_register(pwr_ctrl3),'_X')}")
    print(f"pwr_ctrl4   : 0x{format(read_register(pwr_ctrl4),'_X')}")

    # - Run ten times to average all samples
    for i in range(20):

        out, state, rec = xylo_monitor.evolve(input_data=np.zeros([T, 0]), record_power=True)

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))

    xylo_monitor = []
    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]

for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    print(f'Clock:\t{clock} MHz\nio:\t{io*1000:.1f} mW \t AFE core:\t{analog*1000:.1f} mW\tDFE+SNN core:\t{digital*1000:.1f} mW')

pwr_ctrl1   : 0xEF
pwr_ctrl2   : 0x1_0001
pwr_ctrl3   : 0x0
pwr_ctrl4   : 0x0


## Load a network to the chip

In [ ]:
from rockpool.nn.networks import SynNet
from rockpool.nn.modules import LIFTorch
import warnings
warnings.filterwarnings("ignore")

ckpt = '../../../docs/devices/xylo-a3/model_sample/to_deploy_inXylo.json'

# - Loading trained model architecture parameters
arch_params = {'n_classes': 1,
'n_channels': 16,
'size_hidden_layers':[63, 63, 63],
'time_constants_per_layer':[3,7,7],
'tau_syn_base': 0.02,
'tau_mem': 0.02,
'tau_syn_out': 0.02,
'neuron_model': LIFTorch,
'dt': 0.00994,
'output': 'vmem'}

# - Instantiating the model backbone and loading trained checkpoint
model = SynNet(** arch_params)
model.load(ckpt)

Nin = 16
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper
import rockpool.transform.quantize_methods as q

# getting the model specifications using the mapper function
spec = mapper(model.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
# quantizing the model
spec.update(q.channel_quantize(**spec))

xylo_conf, is_valid, msg = config_from_specification(**spec)

print(xylo_conf)


# Power measurement - idle - with loaded network

In [ ]:
clock_frequencies = [12.5, 25, 50]

In [ ]:
xylo_conf.operation_mode = samna.xyloAudio3.OperationMode.RealTime

dt = 0.01 # seconds
duration = 3 # seconds

T = int(duration/dt) # timesteps

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three second in silence
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=xylo_conf, power_frequency=100, dt=dt, output_mode='Spike')

    sink.clear_events()
    # print(clock)
    # print(f"tr_wrap     : 0x{format(read_register(tr_wrap),'_X')}")
    # pwr_ctrl[1-4]
    print(f"pwr_ctrl1   : 0x{format(read_register(pwr_ctrl1),'_X')}")
    print(f"pwr_ctrl2   : 0x{format(read_register(pwr_ctrl2),'_X')}")
    print(f"pwr_ctrl3   : 0x{format(read_register(pwr_ctrl3),'_X')}")
    print(f"pwr_ctrl4   : 0x{format(read_register(pwr_ctrl4),'_X')}")

    # - Run ten times to average all samples
    for i in range(20):
        out, state, rec = xylo_monitor.evolve(input_data=np.zeros([T, Nin]), record_power=True)

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))

    xylo_monitor = []

    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]


for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    # print(power)
    print(f'Clock:\t{clock} MHz\nio:\t{io*1000:.1f} mW \t AFE core:\t{analog*1000:.1f} mW\tDFE+SNN core:\t{digital*1000:.1f} mW')

# Power measurement - inference over a trained network

In [ ]:
from scipy.io import wavfile
!pip install simpleaudio
import simpleaudio as sa
import numpy as np

def get_wave_object(test_file):
    sample_rate, data = wavfile.read(test_file)

    duration = int(len(data)/sample_rate) # in seconds
    n = data.ndim

    if data.dtype == np.int8:
        bytes_per_sample = 1
    elif data.dtype == np.int16:
        bytes_per_sample = 2
    elif data.dtype == np.float32:
        bytes_per_sample = 4
    else:
        raise ValueError("recorded audio should have 1 or 2 bytes per sample!")

    wave_obj = sa.WaveObject(
        audio_data= data,
        num_channels=data.ndim,
        bytes_per_sample=bytes_per_sample,
        sample_rate=sample_rate
    )

    return duration,wave_obj

In [ ]:
xylo_conf.operation_mode = samna.xyloAudio3.OperationMode.RealTime

dt = 0.01 # seconds
duration = 3 # seconds
T = int(duration/dt) # timesteps

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three seconds playing an audio
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=xylo_conf, power_frequency=100, dt=dt, output_mode='Spike')

    sink.clear_events()
    # print(clock)
    # print(f"tr_wrap     : 0x{format(read_register(tr_wrap),'_X')}")
    # pwr_ctrl[1-4]
    print(f"pwr_ctrl1   : 0x{format(read_register(pwr_ctrl1),'_X')}")
    print(f"pwr_ctrl2   : 0x{format(read_register(pwr_ctrl2),'_X')}")
    print(f"pwr_ctrl3   : 0x{format(read_register(pwr_ctrl3),'_X')}")
    print(f"pwr_ctrl4   : 0x{format(read_register(pwr_ctrl4),'_X')}")

    # - Run ten times to average all samples
    for i in range(5):
        test_audio = '../../../docs/devices/xylo-a3/audio_sample/cry_sample_3sec.wav'
        duration, wave_obj = get_wave_object(test_audio)
        
        play_obj = wave_obj.play()
        out, state, rec = xylo_monitor.evolve(input_data=np.zeros([T, Nin]), record_power=True)
        play_obj.wait_done()
        
        print(f'cry detected: {np.sum(out)>0}')

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))

    xylo_monitor = []

    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]


for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    # print(power)
    print(f'Clock:\t{clock} MHz\nio:\t{io*1000:.1f} mW \t AFE core:\t{analog*1000:.1f} mW\tDFE+SNN core:\t{digital*1000:.1f} mW')